In [1]:
%%init_spark
launcher.jars = ["/app/setup/commons-ip.jar"]
launcher.conf.spark.app.name = "bhavesh_notebook"
launcher.conf.spark.queue = "default"
launcher.conf.spark.local.dir = "/app/tmp"
launcher.conf.spark.sql.shuffle.partitions = 210
launcher.conf.spark.sql.shuffle.minPartitions = 20
launcher.conf.spark.driver.memory = "50g"
launcher.conf.spark.ui.showConsoleProgress = "true"
launcher.master = "local[15]"

In [2]:
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import ai.couture.obelisk.commons.utils.DataFrameUtil.minMaxScaler
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://gcdevctrapp01:4043
SparkContext available as 'sc' (version = 3.3.0, master = local[15], app id = local-1687962779421)
SparkSession available as 'spark'


import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import ai.couture.obelisk.commons.utils.DataFrameUtil.minMaxScaler
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._


In [4]:
var monthlySales = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/temp/queryLevelMonthly")
monthlySales.printSchema

root
 |-- query: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- sales: long (nullable = true)
 |-- distinctPurchases: long (nullable = true)



monthlySales: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 3 more fields]


In [5]:
var predictions = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictions")
predictions.printSchema

root
 |-- similargrouplevel: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = true)



predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, category_name: string ... 8 more fields]


In [5]:
predictions.count

res2: Long = 765221


In [6]:
var historicalActual = monthlySales
// .filter(col("month").isin("2022-06", "2022-07", "2022-08", "2022-09", "2022-10"))
.filter(col("month")>="2022-05" && col("month")<"2023-05")
.groupBy("similargrouplevel", "query")
.pivot("month")
.agg(sum("sales").cast(DoubleType) as "sales")
.na.fill(0)
.select(col("similargrouplevel"), col("query"), col("2022-06") as "m2", col("2022-07") as "m3", 
        col("2022-08") as "m4", col("2022-09") as "m5", col("2022-10") as "m6")
.join(predictions.select("similargrouplevel", "query").distinct, Seq("similargrouplevel", "query"))

historicalActual.printSchema

root
 |-- similargrouplevel: integer (nullable = false)
 |-- query: string (nullable = true)
 |-- m2: double (nullable = false)
 |-- m3: double (nullable = false)
 |-- m4: double (nullable = false)
 |-- m5: double (nullable = false)
 |-- m6: double (nullable = false)



historicalActual: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 5 more fields]


In [7]:
historicalActual.count

res4: Long = 765221


In [8]:
predictions = predictions.withColumn("type", lit("predictions")).drop("category_name")
historicalActual = historicalActual.withColumn("type", lit("historical")).withColumn("m1", lit(0)).withColumn("actual", lit(0)).select(predictions.columns.map(col): _*)
predictions.printSchema
historicalActual.printSchema

root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = true)
 |-- type: string (nullable = false)

root
 |-- similargrouplevel: integer (nullable = false)
 |-- query: string (nullable = true)
 |-- m1: integer (nullable = false)
 |-- m2: double (nullable = false)
 |-- m3: double (nullable = false)
 |-- m4: double (nullable = false)
 |-- m5: double (nullable = false)
 |-- m6: double (nullable = false)
 |-- actual: integer (nullable = false)
 |-- type: string (nullable = false)



predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, query: string ... 8 more fields]
historicalActual: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 8 more fields]


In [10]:
val predictionWeight = 0.5
val historicalWeight = 0.5
Array("m2", "m3", "m4", "m5", "m6").foreach(column => {
    predictions = predictions.withColumn(column, col(column) * lit(predictionWeight))
    historicalActual = historicalActual.withColumn(column, col(column) * lit(historicalWeight))
})

predictionWeight: Double = 0.5
historicalWeight: Double = 0.5


In [11]:
var finalPredictions = predictions.union(historicalActual).groupBy("similargrouplevel", "query")
.agg(
    sum("m1") as "m1",
    sum("m2") as "m2",
    sum("m3") as "m3",
    sum("m4") as "m4",
    sum("m5") as "m5",
    sum("m6") as "m6",
    sum("actual") as "actual"
    )
.withColumn("category_name", when(col("similargrouplevel") === "830216013", lit("Men-Shirts")).when(col("similargrouplevel") === "830303011", lit("Women-Kurtas")))

finalPredictions.printSchema

root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = true)
 |-- category_name: string (nullable = true)



finalPredictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, query: string ... 8 more fields]


In [ ]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/temp/predictionsAvgHistorical", finalPredictions)

In [3]:
var finalPredictions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/temp/predictionsAvgHistorical")
finalPredictions.printSchema
println(finalPredictions.count)

root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = true)
 |-- category_name: string (nullable = true)

765221


finalPredictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, query: string ... 8 more fields]


In [7]:
finalPredictions.filter(col("similargrouplevel") === "830216013").show(5, false)

+-----------------+----------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------+-------------+
|similargrouplevel|query                                               |m1                |m2                |m3                |m4                |m5                |m6                |actual|category_name|
+-----------------+----------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------+-------------+
|830216013        |linen-club_orange_check_2_full-length sleeve_classic|11.477221250534058|13.998111963272095|19.55418872833252 |22.59066939353943 |15.436954975128174|19.67927837371826 |0.0   |Men-Shirts   |
|830216013        |van-heusen_brown_micro-print_3_short sleeve_#       |0.9143150631433059|0.535913178515699 |1.5175595374825264|3.6266786547425056|5.79166609962013  |1

In [9]:
val queries = Array("linen-club_orange_check_2_full-length sleeve_classic", 
                    "van-heusen_brown_micro-print_3_short sleeve_#",
                   "campus-sutra_#_dyed-washed_#_#_indian",
                   "crocodile_blue_leaf_#_full-length sleeve_#",
                   "tistabene_teal_abstract_#_#_indian")
predictions.filter(col("similargrouplevel") === "830216013").filter(col("query").isin(queries: _*)).show(false)

+-----------------+-------------+----------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------+
|similargrouplevel|category_name|query                                               |m1                |m2                |m3                |m4                |m5                |m6                |actual|
+-----------------+-------------+----------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------+
|830216013        |Men-Shirts   |linen-club_orange_check_2_full-length sleeve_classic|11.477221250534058|14.99622392654419 |19.10837745666504 |17.181338787078857|19.873909950256348|22.358556747436523|0.0   |
|830216013        |Men-Shirts   |campus-sutra_#_dyed-washed_#_#_indian               |2.0568788829259574|2.384276587050408 |6.094819504301995 |4.479139763396233 |4.6177

queries: Array[String] = Array(linen-club_orange_check_2_full-length sleeve_classic, van-heusen_brown_micro-print_3_short sleeve_#, campus-sutra_#_dyed-washed_#_#_indian, crocodile_blue_leaf_#_full-length sleeve_#, tistabene_teal_abstract_#_#_indian)


In [11]:
monthlySales.filter(col("similargrouplevel") === "830216013").filter(col("query").isin(queries: _*)).groupBy("similargrouplevel", "query").pivot("month").agg(sum("sales")).show(false)

+-----------------+----------------------------------------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|similargrouplevel|query                                               |2022-01|2022-02|2022-03|2022-04|2022-05|2022-06|2022-07|2022-08|2022-09|2022-10|2022-11|2022-12|2023-01|2023-02|2023-03|2023-04|2023-05|2023-06|
+-----------------+----------------------------------------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|830216013        |campus-sutra_#_dyed-washed_#_#_indian               |1      |1      |5      |1      |null   |null   |null   |1      |null   |null   |1      |1      |null   |null   |1      |null   |1      |1      |
|830216013        |van-heusen_brown_micro-print_3_short sleeve_#       |null   |null   |null   |null   |null   |null   |1      |6   

In [12]:
var top20WKQ = CSVToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/temp/topQueries/WomenKurtas/top20", inferSchema=true).select("similargrouplevel", "query")
top20WKQ.printSchema

root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)



top20WKQ: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string]


In [ ]:
var trimmedPredictions = finalPredictions.join(top20WKQ, Seq("similargrouplevel", "query"))
trimmedPredictions.printSchema
println(trimmedPredictions.count)
DFToCSV.putDF("/data/Archive/bhavesh/InventoryPrediction/temp/womenKurtasTop20Predictions", trimmedPredictions.coalesce(1))

root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = true)
 |-- category_name: string (nullable = true)

20


In [7]:
var df = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/experiments/exp_neural_network_dropout_lr_0.0005/metricsActualDF/productLevelModelForEachBrick/queryLevel/model=neural_network/date=2023-05-01")
df.printSchema

df = df.withColumn("absError", abs(col("actual") - col("predicted")))
df = df.withColumn("errorRank", row_number().over(Window.partitionBy("similargrouplevel").orderBy(desc("absError"))))

val limitQueries = 5
df = df.filter(col("errorRank")<=limitQueries).select("similargrouplevel", "query", "template")

var oldQueryToNewQueryMap = ParquetToDF.getDF("/data/Archive/inventory/2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/attributesQueryInfo/final").select("query","template","similargrouplevel","newQuery")
df = df.join(oldQueryToNewQueryMap, Seq("similargrouplevel", "query", "template")).drop("query", "template").withColumnRenamed("newQuery", "query")
df.printSchema

var queryProductMap = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/QueryProductMapping-Revamp/suffix=colorfamily_pricebucket_styletype_pattern_sleeve_brandname").drop("template")

var highErrorQueries = df.join(queryProductMap, Seq("similargrouplevel", "query"))
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/temp/highErrorQueries", highErrorQueries)

root
 |-- query: string (nullable = true)
 |-- template: string (nullable = true)
 |-- actual: long (nullable = true)
 |-- predicted: double (nullable = true)
 |-- lowerBound: double (nullable = true)
 |-- upperBound: double (nullable = true)
 |-- similargrouplevel: integer (nullable = true)

root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)



df: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string]
df: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string]
df: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string]
limitQueries: Int = 5
df: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string]
oldQueryToNewQueryMap: org.apache.spark.sql.DataFrame = [query: string, template: string ... 2 more fields]
df: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string]
queryProductMap: org.apache.spark.sql.DataFrame = [similargrouplevel: int, productid: string ... 1 more field]
highErrorQueries: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 1 more field]
